# Project to map Toronto neighborhoods#
## Initialize ##

In [1]:
import numpy as np
import pandas as pd
import json
#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

print('Libraries imported.')

Libraries imported.


## Read the webpage using standard panda function. Page inspection revealed that the first table is our target, hence first element ##

In [2]:
dfs = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', header=0)
dfs[0].head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


### Drop all "Not assigned" Boroughs ###

In [3]:
Toronto=dfs[0]
Toronto=Toronto[Toronto['Borough']!="Not assigned"].reset_index(drop=True)
Toronto.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


### Checked to see if any "Not assigned" neighborhoods. Not in our table, so we move on to print the shape ###

In [4]:
#print(Toronto[Toronto['Neighbourhood']=="Not assigned"])
print("Table Size:",Toronto.shape)


Table Size: (103, 3)


## Geocoder Call didn't work. Had to interrupt the kernel a few times ##

In [5]:
#!pip install geocoder
"""import geocoder
latitude=[]
longitude=[]
i=0
#for i in range(0,len(Toronto['Postal Code']))
# initialize your variable to None
lat_lng_coords = None
postal_code=Toronto['Postal Code'][i]
print(Toronto['Postal Code'][i])
# loop until you get the coordinates
while(lat_lng_coords is None):
    g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
    lat_lng_coords = g.latlng
    #print(lat_lng_coords)

latitude[i] = lat_lng_coords[0]
longitude[i] = lat_lng_coords[1]
print(latitude[i])"""

"import geocoder\nlatitude=[]\nlongitude=[]\ni=0\n#for i in range(0,len(Toronto['Postal Code']))\n# initialize your variable to None\nlat_lng_coords = None\npostal_code=Toronto['Postal Code'][i]\nprint(Toronto['Postal Code'][i])\n# loop until you get the coordinates\nwhile(lat_lng_coords is None):\n    g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))\n    lat_lng_coords = g.latlng\n    #print(lat_lng_coords)\n\nlatitude[i] = lat_lng_coords[0]\nlongitude[i] = lat_lng_coords[1]\nprint(latitude[i])"

### Hence I used the csv file provided ###

In [6]:
df_latlon = pd.read_csv('http://cocl.us/Geospatial_data')
df_latlon.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Merged the two dataframes using inner merge to keep the rows consistent with our neighborhood data ###

In [12]:
df_final=pd.merge(Toronto, df_latlon, how='inner', left_on = 'Postal Code', right_on = 'Postal Code')
print("Table Size: \n",df_final.shape)
df_final.head()


Table Size: 
 (103, 5)


,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
